In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xarray as xr
from pathlib import Path
import glob
from data_locations import LOC_FORECASTS_SI, LOC_OBSERVATIONS_SI
from tqdm import tqdm
# from preprocessing import get_anomalies #, get_climatology 
from preprocessing import align_data_and_targets
from preprocessing import AnomaliesScaler, Detrender, Standardizer, Normalizer, PreprocessingPipeline, Spatialnanremove, create_mask
from preprocessing import detrend, standardize, normalize, linear_debiasing

In [ ]:
var     = 'SICN'
fct_set = 'canesm5'    
NPSproj = True
if NPSproj:
    crs = 'NPS'  
else: 
    crs = '1x1'

In [6]:
# specify data directories --specified in data_locations.py
data_dir_forecast = LOC_FORECASTS_SI
data_dir_obs = glob.glob(LOC_OBSERVATIONS_SI+ f'/NASA*{crs}*.nc')[0]

In [ ]:

print("Load observations")
obs_in = xr.open_dataset(data_dir_obs)[var].clip(0,1)
print("Load forecasts") 
ls = [xr.open_dataset(glob.glob(LOC_FORECASTS_SI + f'/*_initial_month_{intial_month}_*{crs}*.nc')[0])['SICN'].mean('ensembles').load() for intial_month in range(1,13) ]
ds_in = xr.concat(ls, dim = 'time').sortby('time').clip(0,1)

In [ ]:
# out_dir = '/space/hall5/sitestore/eccc/crd/ccrn/users/rpg002/output/SI/Full/results/CNN/run_set_1/N15_M12_v1_North_batch100_e20/'
# ds_in =  xr.open_mfdataset(str(Path(out_dir, "*.nc")), combine='nested', concat_dim='time').load()['nn_adjusted']

In [ ]:
fct, obs = align_data_and_targets(ds_in, obs_in, 12)
if not fct.time.equals(obs.time):
    fct = fct.sel(time = obs.time)
assert fct.time.equals(obs.time)

In [7]:
y0_test = 2006
y1_test = 2020

In [9]:
dir_frnt_gen     = '/space/hall5/sitestore/eccc/crd/ccrn/users/rpg002/output/SI/Full/results/NASA'

dir_badj_out = f'{dir_frnt_gen}/Bias_Adjusted/'
dir_tadj_out = f'{dir_frnt_gen}/Trend_Adjusted/'
dir_ladj_out = f'{dir_frnt_gen}/Linear_Adjusted/'

Path(dir_badj_out).mkdir(parents=True,exist_ok=True)
Path(dir_tadj_out).mkdir(parents=True,exist_ok=True)
Path(dir_ladj_out).mkdir(parents=True,exist_ok=True)

# Bias adjustment

In [ ]:
def calculate_climatology(ds):
    return xr.concat([ds.isel(time = np.arange(0,len(ds.time),12) + init_month).mean('time') for init_month in range(12) ], dim = 'init_month').assign_coords(init_month = np.arange(1,13))

def calculate_anomalies(ds, monthly_climatology):
    return xr.concat([ds.where(np.mod(ds.time,100) == init_month, drop = True) - monthly_climatology.sel(init_month = init_month).values for init_month in range(1,13)], dim = 'time').sortby('time')

def bias_adj(fct, fct_clim, obs_clim):
        fct_anom  = calculate_anomalies(fct, fct_clim)
        return xr.concat([fct_anom.where(np.mod(fct_anom.time,100) == init_month, drop = True) + obs_clim.sel(init_month = init_month).values for init_month in range(1,13)], dim = 'time').sortby('time')


In [10]:
#bias adjustment


test_years = np.arange(y0_test, y1_test + 1 )

ls = []

for y_idx, test_year in enumerate(test_years):
    print(f"bias adjustment: test year {test_year}") 
    ds_full  = fct.where(fct.time < 100* (test_year + 1), drop  =True)  ### or fct_anom
    obs_full = obs.where(obs.time < 100* (test_year + 1), drop  =True) ### or obs_anom
    ds_base = ds_full.where(ds_full.time < 100 * test_year, drop  =True)
    obs_base = obs_full.where(obs_full.time < 100 * test_year, drop  =True)
    train_mask = create_mask(ds_base)  
    preprocessing_mask = np.broadcast_to(train_mask[...,None,None], ds_base.shape)
    fct_clim = calculate_climatology(ds_base.where(~preprocessing_mask))
    obs_clim = calculate_climatology(obs_base.where(~preprocessing_mask))
    fct_debiased = bias_adj(ds_full , fct_clim , obs_clim)
    ls.append(fct_debiased.where(fct_debiased.time > test_year*100 ,drop = True) )
    
fct_debiased = xr.concat(ls,
                         dim='time')

bias adjustment: test year 1995
bias adjustment: test year 1996
bias adjustment: test year 1997
bias adjustment: test year 1998
bias adjustment: test year 1999
bias adjustment: test year 2000
bias adjustment: test year 2001
bias adjustment: test year 2002
bias adjustment: test year 2003
bias adjustment: test year 2004
bias adjustment: test year 2005
bias adjustment: test year 2006
bias adjustment: test year 2007
bias adjustment: test year 2008
bias adjustment: test year 2009
bias adjustment: test year 2010
bias adjustment: test year 2011
bias adjustment: test year 2012
bias adjustment: test year 2013
bias adjustment: test year 2014
bias adjustment: test year 2015
bias adjustment: test year 2016
bias adjustment: test year 2017
bias adjustment: test year 2018
bias adjustment: test year 2019


In [1]:
fct_debiased.to_netcdf(f'{dir_badj_out}/bias_adjusted_{y0_test}-{y1_test}_NPSproj.nc')

NameError: name 'fct_debiased' is not defined

# Trend adjustment

In [ ]:
def calculator_detrender(ds_base):
    detrender = Detrender(trend_dim='time', deg=1)
    detrenders = [ detrender.fit(ds_base.isel(time = np.arange(0,len(ds_base.time),12) + init_month)) for init_month in range(12) ]   
    return detrenders


def trend_adj(ds, fct_detrender, obs_detrender): 
    fct_detrended = [fct_detrender[init_month].transform(ds.isel(time = np.arange(0,len(ds.time),12) + init_month),  remove_intercept=True) for init_month in range(12)]  
    return xr.concat([obs_detrender[init_month].inverse_transform(fct_detrended[init_month],  add_intercept=True) for init_month in range(12)] , dim = 'time').sortby('time')
    


In [12]:
#trend adjustment

test_years = np.arange(y0_test, y1_test + 1 )

ls = []

for y_idx, test_year in tqdm(enumerate(test_years)):
    
    print(f"trend adjustment: test year {test_year}")

    ds_full  = fct.where(fct.time < 100* (test_year + 1), drop  =True)  ### or fct_anom
    obs_full = obs.where(obs.time < 100* (test_year + 1), drop  =True) ### or obs_anom

    ds_base = ds_full.where(ds_full.time < 100 * test_year, drop  =True)
    obs_base = obs_full.where(obs_full.time < 100 * test_year, drop  =True)
    
    train_mask = create_mask(ds_base)
    preprocessing_mask = np.broadcast_to(train_mask[...,None,None], ds_base.shape)
 
    fct_detrender = calculator_detrender(ds_base.where(~preprocessing_mask))
    obs_detrender = calculator_detrender(obs_base.where(~preprocessing_mask))

    fct_tr_adj = trend_adj(ds_full , fct_detrender , obs_detrender)
    
    ls.append(fct_tr_adj.where(fct_tr_adj.time > test_year*100 ,drop = True))

fct_trendcorr = xr.concat(ls, dim='time')

trend adjustment: test year 1995
trend adjustment: test year 1996
trend adjustment: test year 1997
trend adjustment: test year 1998
trend adjustment: test year 1999
trend adjustment: test year 2000
trend adjustment: test year 2001
trend adjustment: test year 2002
trend adjustment: test year 2003
trend adjustment: test year 2004
trend adjustment: test year 2005
trend adjustment: test year 2006
trend adjustment: test year 2007
trend adjustment: test year 2008
trend adjustment: test year 2009
trend adjustment: test year 2010
trend adjustment: test year 2011
trend adjustment: test year 2012
trend adjustment: test year 2013
trend adjustment: test year 2014
trend adjustment: test year 2015
trend adjustment: test year 2016
trend adjustment: test year 2017
trend adjustment: test year 2018
trend adjustment: test year 2019


In [13]:
fct_trendcorr.to_netcdf(f'{dir_tadj_out}/trend_adjusted_{y0_test}-{y1_test}.nc')